In [ ]:
!pip install tweepy
!pip install kafka-python
!wget https://dlcdn.apache.org/kafka/3.0.0/kafka_2.13-3.0.0.tgz
!tar -xzf kafka_2.13-3.0.0.tgz

     |████████████████████████████████| 246 kB 15.8 MB/s 
--2021-11-22 11:22:10--  https://dlcdn.apache.org/kafka/3.0.0/kafka_2.13-3.0.0.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86396520 (82M) [application/x-gzip]
Saving to: ‘kafka_2.13-3.0.0.tgz’

kafka_2.13-3.0.0.tg 100%[===================>]  82.39M   215MB/s    in 0.4s    

2021-11-22 11:22:11 (215 MB/s) - ‘kafka_2.13-3.0.0.tgz’ saved [86396520/86396520]



In [ ]:
import json

import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import StreamListener
from kafka import KafkaProducer, KafkaConsumer
import time
from datetime import datetime, timedelta

In [ ]:
!./kafka_2.13-3.0.0/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.0.0/config/zookeeper.properties
!./kafka_2.13-3.0.0/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.0.0/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


In [ ]:
!ps -ef | grep kafka

root         446       1 17 11:22 ?        00:00:02 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -XX:MaxInlineLevel=15 -Djava.awt.headless=true -Xlog:gc*:file=/content/kafka_2.13-3.0.0/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=100M -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/kafka_2.13-3.0.0/bin/../logs -Dlog4j.configuration=file:./kafka_2.13-3.0.0/bin/../config/log4j.properties -cp /content/kafka_2.13-3.0.0/bin/../libs/activation-1.1.1.jar:/content/kafka_2.13-3.0.0/bin/../libs/aopalliance-repackaged-2.6.1.jar:/content/kafka_2.13-3.0.0/bin/../libs/argparse4j-0.7.0.jar:/content/kafka_2.13-3.0.0/bin/../libs/audience-annotations-0.5.0.jar:/content/kafka_2.13-3.0.0/bin/../libs/commons-cli-1.4.jar:/content/kafka_2.13-3.0.0/bin/../libs/commons-lang3-3.8.1.jar:/content/kafka_2.13

In [ ]:
!./kafka_2.13-3.0.0/bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 3 --topic tugas-11-2

Created topic tugas-11-2.


In [ ]:
!./kafka_2.13-3.0.0/bin/kafka-topics.sh --describe --bootstrap-server localhost:9092 --topic tugas-11-2

Topic: tugas-11-2	TopicId: KNDpblALQcCtsc_6jaMT2g	PartitionCount: 3	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: tugas-11-2	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
	Topic: tugas-11-2	Partition: 1	Leader: 0	Replicas: 0	Isr: 0
	Topic: tugas-11-2	Partition: 2	Leader: 0	Replicas: 0	Isr: 0


In [ ]:
def normalize_timestamp(time):
    mytime = datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    mytime += timedelta(hours=1)
    return (mytime.strftime('%Y-%m-%d %H:%M:%S'))

def on_send_success(record_metadata):
    # print(record_metadata.topic)
    print(record_metadata)
    # print(record_metadata.offset)

def on_send_error(excp):
    log.error('I am an errback', exc_info=excp)

In [ ]:
class Listener(StreamListener):
  def __init__(self, time_limit=60):
    self.start_time = time.time()
    self.limit = time_limit
    self.saveFile = open('tweet.json', 'a')
    super(Listener, self).__init__()

  def on_data(self, data):
    if (time.time() - self.start_time) < self.limit:
      self.saveFile.write(data)
      self.saveFile.write('\n')
      producer.send(topic_name, data).add_callback(on_send_success).add_errback(on_send_error)
      return True
    else:
      self.saveFile.close()
      return False

In [ ]:
consumer_key = "TCHVbt5fbcttUMjNoNYPLsyzZ"
consumer_secret = "wih7b5RUNRkezZouANXqm1aXfTq6A9SOcIcQxJ4WY5tW1b5WT0"
access_token = "1000362437558743041-VnnlqzolYZfdp843j3ZaLVZ1oAnbiK"
access_token_secret = "lmqiLv45HT3ckrnLAz0XaYdsBamkxzpszDRpbS9NeLAT5"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [ ]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],api_version=(3, 0, 0), value_serializer=lambda x: x.encode('utf-8'))
topic_name = 'tugas-11-2'

In [ ]:
stream = Stream(auth, Listener(time_limit=1800))
stream.filter(track=['Shopee'])

RecordMetadata(topic='tugas-11-2', partition=0, topic_partition=TopicPartition(topic='tugas-11-2', partition=0), offset=0, timestamp=1637580153200, log_start_offset=0, checksum=None, serialized_key_size=-1, serialized_value_size=8655, serialized_header_size=-1)
RecordMetadata(topic='tugas-11-2', partition=1, topic_partition=TopicPartition(topic='tugas-11-2', partition=1), offset=0, timestamp=1637580153357, log_start_offset=0, checksum=None, serialized_key_size=-1, serialized_value_size=8665, serialized_header_size=-1)
RecordMetadata(topic='tugas-11-2', partition=2, topic_partition=TopicPartition(topic='tugas-11-2', partition=2), offset=0, timestamp=1637580154440, log_start_offset=0, checksum=None, serialized_key_size=-1, serialized_value_size=3680, serialized_header_size=-1)
RecordMetadata(topic='tugas-11-2', partition=2, topic_partition=TopicPartition(topic='tugas-11-2', partition=2), offset=1, timestamp=1637580155744, log_start_offset=0, checksum=None, serialized_key_size=-1, seriali

In [ ]:
import sqlite3
try:
  sqliteConnection = sqlite3.connect('Tweets.db')
  cursor = sqliteConnection.cursor()
  print("Database created and Successfully Connected to SQLite")

  query = "select sqlite_version();"
  cursor.execute(query)
  record = cursor.fetchall()
  print("SQLite Database Version is: ", record)

  query = """CREATE TABLE tweet 
      (id TEXT PRIMARY KEY NOT NULL,
      user_id TEXT NOT NULL,
      username TEXT NOT NULL,
      tweet TEXT NOT NULL,
      created_at TEXT NOT NULL,
      favorite_count INTEGER NOT NULL,
      retweet_count INTEGER NOT NULL,
      reply_count INTEGER NOT NULL,
      quote_count INTEGER NOT NULL,
      lang_type TEXT
      );"""
  cursor.execute(query)
  sqliteConnection.commit()
  cursor.close()

except sqlite3.Error as error:
  print("Error while connecting to sqlite", error)
finally:
  if sqliteConnection:
    sqliteConnection.close()
    print("The SQLite connection is closed")

Database created and Successfully Connected to SQLite
SQLite Database Version is:  [('3.22.0',)]
The SQLite connection is closed


In [ ]:
def load(msg):
  temp = json.loads(msg)
  tweet = (temp['id'],temp['user']['id'],temp['user']['name'], temp['created_at'],
           temp['favorite_count'], temp['retweet_count'], temp['reply_count'],
           temp['quote_count'], temp['lang'], temp['text'])
  try:
    sqliteConnection = sqlite3.connect('Tweets.db')
    
    cursor = sqliteConnection.cursor()
    query = f"""INSERT INTO tweet (id, user_id, username, 
                created_at, favorite_count, retweet_count, reply_count, quote_count,
                lang_type, tweet)
            VALUES
              (?,?,?,?,?,?,?,?,?,?);"""
    cursor.execute(query, tweet)
    sqliteConnection.commit()
    cursor.close()

  except sqlite3.Error as error:
    print("Error while connecting to sqlite", error)
  finally:
    if sqliteConnection:
      sqliteConnection.close()  

In [ ]:
consumer1 = KafkaConsumer('tugas-11-2', bootstrap_servers=['localhost:9092'],
                        auto_offset_reset='earliest',
                        enable_auto_commit=True,
                        group_id='consumers',
                        value_deserializer=lambda x: x.decode('utf8'), 
                        consumer_timeout_ms=1800000,
                        api_version=(3, 0, 0))
# consumer2 = KafkaConsumer('tugas-11-2', bootstrap_servers=['localhost:9092'],
#                         auto_offset_reset='earliest',
#                         enable_auto_commit=True,
#                         group_id='consumers',
#                         value_deserializer=lambda x: x.decode('utf8'), 
#                         consumer_timeout_ms=10000,
#                         api_version=(3, 0, 0))
# consumer3 = KafkaConsumer('tugas-11-2', bootstrap_servers=['localhost:9092'],
#                         auto_offset_reset='earliest',
#                         enable_auto_commit=True,
#                         group_id='consumers',
#                         value_deserializer=lambda x: x.decode('utf8'), 
#                         consumer_timeout_ms=10000,
#                         api_version=(3, 0, 0))
# print(consumer2.poll(timeout_ms=60000))
for msg in consumer1:
  load(msg.value)
# for msg in consumer2:
#   load(msg.value)
# for msg in consumer3:
#   load(msg.value)